### Question

You selected Rutgers-Newark for various reasons (e.g. the DS Minor or its close to home). As a non-
US citizen, I am unfamiliar with US higher education. However, my closest friends have attended
Rutgers-Newark and have told me that it provides a good return on investment. As much as I like my
friends, I do not think that Rutgers-Newark is the average American college.

Null Hypothesis H 0 : Rutgers University Newark is consistently representative of the average US
college/university

Data: MERGED2019_17_pp.csv (AWS S3 bucket) and Data Dictionary

In [1]:
# Holds all imports
import pandas as pd
from IPython.display import display

In [2]:
# Reads and prints csv
base_data = pd.read_csv("MERGED2019_17_pp.csv", low_memory=False)
base_data

,Unnamed: 0,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,...,FTFTPCTPELL,FTFTPCTFLOAN,UG12MN,G12MN,SCUGFFN,POOLYRS_FTFTAIDPCT,FTFTPCTPELL_POOLED_SUPP,FTFTPCTFLOAN_POOLED_SUPP,SCUGFFN_POOLED,YEAR
0,9,100654,100200,1002,Alabama A & M University,Normal,AL,35762,NaN,NaN,...,0.7373,0.7529,4865.0,1292.0,1473.0,NaN,NaN,NaN,NaN,2019
1,0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,...,0.7057,0.7143,5343.0,1165.0,1288.0,2.0,0.7083,0.7287,2698.0,2017
2,0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,NaN,NaN,...,0.7106,0.7418,5207.0,1185.0,1410.0,NaN,NaN,NaN,NaN,2018
3,10,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,NaN,NaN,...,0.3415,0.4934,13440.0,8114.0,1593.0,NaN,NaN,NaN,NaN,2019
4,1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,NaN,NaN,...,0.4009,0.5488,13308.0,9888.0,1948.0,NaN,NaN,NaN,NaN,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21151,7110,48778201,4221501,42215,BCI - Malden,Malden,MA,021480000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
21152,3000,48778201,4221501,42215,BCI - Malden,Malden,MA,21480000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
21153,6804,49005401,4182601,41826,HCI College - Fort Lauderdale Campus,Fort Lauderdale,FL,33309,Accrediting Commission of Career Schools and C...,www.hci.edu/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017
21154,7111,49005401,4182601,41826,HCI College - Fort Lauderdale Campus,Fort Lauderdale,FL,33309,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018


## Preliminary Goals ##

**Null Hypothesis:**  Rutgers University Newark is consistently representative of the average US college/university.

After looking through the data dictionary and the data in excel, there are few things that were noticed...
    1. The data has many duplicated rows. Colleges in the data in multiple rows.
    2. There are also colleges that are not operational that appear in the data.
    3. The sources for these values are government websites, that means that the data is more trustworthy.

Goals
    1. Clean up the data by...
        a. Making the subset smaller with columns that we are most interested in
        b. Convert that subset into a new subset with only row for each college
        c. Remove the non operating colleges because the business use case is concerned with if Rutgers University is currently a good investment and because non operating colleges don't match current college standards, they will skew the results and bring bias

Interesting Columns: After looking through the data dictionary these columns were good columns to dive deeper into for the results we are looking for...
    1. CURROPER
            a. This notes if the collegs are currently operational. This is important because we only care for colleges that are operational
    2. INSTNM
        a. This gives us the college's name which will be a useful identifier during final analysis when we are examining the data
    3. OPEID
        a. It is an unique id for each college. This is a more reliable identifier than names for data manipulation because integers are easier to work with in terms of code
    4. LOCALE
        a. It gives us a general idea of the type of location that the college is located in. For example is it an urban community or a rural community where the college resides. This helps with assessing campus life and student life. Environments impact colleges heavily
    5. CONTROL
        a. Checks if the college is private non-profit, private profit, or public. This classification impacts a lot of other traits for colleges like expenses so it is vital to look at
    6. CCUGPROF
        a. Gives us a generic profile of the college. This categorization of colleges lets us evaluation what the typical type of college looks like. For example is it a two year college or a four year college
    7. CCSIZSET
        a. Gives us the size of the college. This lets us evaluate if the average college in the US is big or small
    8. ADM_RATE
        a. Gives us a generic idea of how difficult it is to attend the college
    9. SAT_AVG
        a. Gives more context on the difficulty of attending the college
    10. TUITIONFEE_IN, TUITIONFEE_OUT
        a. Gives us the cost to attend the college
    11. C150_4, C150_L4
        a. Gives us the compelition rate for getting a undergrad degree in these colleges. This is a metric to evaluate return investment where the return is obtaining a degree within time
        

In [3]:
# Makes a subset of only desired columns
sub_test = base_data[['OPEID', 'INSTNM', 'CURROPER', 'LOCALE', 'CONTROL', 'CCUGPROF', 'CCSIZSET', 'ADM_RATE', 'SAT_AVG', 'TUITIONFEE_IN', 'TUITIONFEE_OUT', 'C150_4', 'C150_L4']]
sub_test

,OPEID,INSTNM,CURROPER,LOCALE,CONTROL,CCUGPROF,CCSIZSET,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,C150_4,C150_L4
0,100200,Alabama A & M University,NaN,NaN,1,NaN,NaN,0.8738,849.0,9366.0,17496.0,0.2749,NaN
1,100200,Alabama A & M University,1.0,12.0,1,10.0,14.0,0.8986,957.0,9744.0,18354.0,0.2685,NaN
2,100200,Alabama A & M University,NaN,NaN,1,NaN,NaN,0.9027,929.0,9857.0,18236.0,0.2396,NaN
3,105200,University of Alabama at Birmingham,NaN,NaN,1,NaN,NaN,0.5814,1125.0,8040.0,18368.0,0.5309,NaN
4,105200,University of Alabama at Birmingham,NaN,NaN,1,NaN,NaN,0.9181,1195.0,8328.0,19032.0,0.5292,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21151,4221501,BCI - Malden,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21152,4221501,BCI - Malden,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21153,4182601,HCI College - Fort Lauderdale Campus,1.0,NaN,3,NaN,NaN,NaN,NaN,21420.0,NaN,NaN,NaN
21154,4182601,HCI College - Fort Lauderdale Campus,NaN,NaN,3,NaN,NaN,NaN,NaN,22575.0,NaN,NaN,NaN


In [4]:
# Obtains total number of unique colleges based on id
print("NUM OF NULL IDS: " + str(sub_test['OPEID'].isnull().sum()))
print("NUM OF UNIQUES: " + str(len(list(sub_test['OPEID'].unique()))))
print("NUM OF NON NULL OPER VALUES: " + str(sub_test['CURROPER'].value_counts().sum()))

NUM OF NULL IDS: 0
NUM OF UNIQUES: 7550
NUM OF NON NULL OPER VALUES: 6806


## Data Cleaning ##

To clean the data, we will seperate each columns using id as the independent variable and the column as the dependent. For each of these columns we will drop the data points that are not avaliable so that each subset for the column only holds values that do exist. Then after this is done, we will assess if there are any duplicate rows based on OPEID and if there are, we will best convert them into one row based on the column. Once that is done, we will merge all the column subsets into one big subset where each college row holds all its data and there are no duplicates!

In [5]:
# Defines function for creating column subsets
def col_sub(col_name):
    # Makes subset
    sub = sub_test[['OPEID', col_name]].dropna()
    
    # Prints subset
    print(col_name + " SUBSET")
    display(sub)
    print("---------------------------")
    
    # Returns subset
    return sub

In [6]:
# Creates a subset for each column
name_sub = col_sub('INSTNM')
oper_sub = col_sub('CURROPER')
locale_sub = col_sub('LOCALE')
control_sub = col_sub('CONTROL')
ccugprof_sub = col_sub('CCUGPROF')
ccsizset_sub = col_sub('CCSIZSET')
adm_rate_sub = col_sub('ADM_RATE')
sat_avg_sub = col_sub('SAT_AVG')
tut_in_sub = col_sub('TUITIONFEE_IN')
tut_out_sub = col_sub('TUITIONFEE_OUT')
comp_yr4_sub = col_sub('C150_4')
comp_lyr4_sub = col_sub('C150_L4')

INSTNM SUBSET


,OPEID,INSTNM
0,100200,Alabama A & M University
1,100200,Alabama A & M University
2,100200,Alabama A & M University
3,105200,University of Alabama at Birmingham
4,105200,University of Alabama at Birmingham
...,...,...
21151,4221501,BCI - Malden
21152,4221501,BCI - Malden
21153,4182601,HCI College - Fort Lauderdale Campus
21154,4182601,HCI College - Fort Lauderdale Campus


---------------------------
CURROPER SUBSET


,OPEID,CURROPER
1,100200,1.0
5,105200,1.0
6,2503400,1.0
11,105500,1.0
13,100500,1.0
...,...,...
21148,157114,1.0
21149,157115,1.0
21150,4204801,1.0
21153,4182601,1.0


---------------------------
LOCALE SUBSET


...

---------------------------
C150_4 SUBSET


,OPEID,C150_4
0,100200,0.2749
1,100200,0.2685
2,100200,0.2396
3,105200,0.5309
4,105200,0.5292
...,...,...
19722,4250000,0.2500
19731,4265200,0.1111
19767,4276100,0.1778
19787,246100,0.1890


---------------------------
C150_L4 SUBSET


,OPEID,C150_L4
18,100700,0.2832
19,100700,0.2784
20,100700,0.2153
33,1218200,0.1108
34,1218200,0.1958
...,...,...
19766,4277400,1.0000
19768,4275200,1.0000
19770,4223501,0.7813
19780,4276800,0.8000


---------------------------


In [7]:
# Removes duplicates for name
name_sub.drop_duplicates(subset=['OPEID'], keep='first', inplace=True)
name_sub

,OPEID,INSTNM
0,100200,Alabama A & M University
3,105200,University of Alabama at Birmingham
6,2503400,Amridge University
9,105500,University of Alabama in Huntsville
12,100500,Alabama State University
...,...,...
21149,157115,Georgia Military College - Eastman
21150,4204801,American College of Barbering - Florence
21151,4221501,BCI - Malden
21153,4182601,HCI College - Fort Lauderdale Campus


In [8]:
# Checks for duplicates in operation
print("NUM OF DUPLICATES IDs: " + str(oper_sub.duplicated(subset=['OPEID']).sum()))
print("NUM OF DUPLICATE ROWS: " + str(oper_sub.duplicated().sum()))

NUM OF DUPLICATES IDs: 30
NUM OF DUPLICATE ROWS: 30


In [9]:
# Removes all duplicates because all rows that are duplicate also contain the same values
oper_sub.drop_duplicates(subset=['OPEID'], inplace=True)
oper_sub

,OPEID,CURROPER
1,100200,1.0
5,105200,1.0
6,2503400,1.0
11,105500,1.0
13,100500,1.0
...,...,...
21148,157114,1.0
21149,157115,1.0
21150,4204801,1.0
21153,4182601,1.0


In [10]:
# Checks for duplicates in locale
print("NUM OF DUPLICATES IDs: " + str(locale_sub.duplicated(subset=['OPEID']).sum()))
print("NUM OF DUPLICATE ROWS: " + str(oper_sub.duplicated().sum()))

NUM OF DUPLICATES IDs: 25
NUM OF DUPLICATE ROWS: 0


In [11]:
# Gets a temporary subset of all duplicates
locale_dup = locale_sub[locale_sub.duplicated(subset=['OPEID'], keep=False)]
locale_dup.sort_values(['OPEID'])

,OPEID,LOCALE
1937,145900,11.0
17072,145900,11.0
2044,146900,13.0
18091,146900,13.0
2013,147900,13.0
13872,147900,13.0
2075,153500,12.0
18732,153500,12.0
19790,180500,11.0
3298,180500,11.0


In [12]:
# There is only one case where the value values are different between 
# duplicate ids so dropping one of them will not impact the data much
locale_sub.drop_duplicates(subset=['OPEID'], inplace=True)
locale_sub

,OPEID,LOCALE
1,100200,12.0
5,105200,12.0
6,2503400,12.0
11,105500,12.0
13,100500,12.0
...,...,...
19794,4238601,21.0
19795,3889301,21.0
19796,4162401,21.0
19797,3915304,23.0


In [13]:
# Checks for duplicates in control
print("NUM OF DUPLICATES IDs: " + str(control_sub.duplicated(subset=['OPEID']).sum()))
print("NUM OF DUPLICATE ROWS: " + str(control_sub.duplicated().sum()))

NUM OF DUPLICATES IDs: 13606
NUM OF DUPLICATE ROWS: 13574


In [14]:
# Most rows contain the same values as their duplicates
control_sub.drop_duplicates(subset=['OPEID'], inplace=True)
control_sub

,OPEID,CONTROL
0,100200,1
3,105200,1
6,2503400,2
9,105500,1
12,100500,1
...,...,...
21149,157115,1
21150,4204801,3
21151,4221501,2
21153,4182601,3


In [15]:
# Checks for duplicates in CCUGPROF
print("NUM OF DUPLICATES IDs: " + str(ccugprof_sub.duplicated(subset=['OPEID']).sum()))
print("NUM OF DUPLICATE ROWS: " + str(ccugprof_sub.duplicated().sum()))

NUM OF DUPLICATES IDs: 25
NUM OF DUPLICATE ROWS: 3


In [16]:
# Gets a temporary subset of all duplicates
ccugprof_dup = ccugprof_sub[ccugprof_sub.duplicated(subset=['OPEID'], keep=False)]
ccugprof_dup.sort_values(['OPEID'])

,OPEID,CCUGPROF
1937,145900,5.0
17072,145900,5.0
2044,146900,15.0
18091,146900,5.0
2013,147900,14.0
13872,147900,5.0
2075,153500,15.0
18732,153500,5.0
19790,180500,-2.0
3298,180500,7.0


In [17]:
# Removes duplicates
ccugprof_sub.drop_duplicates(subset=['OPEID'], inplace=True)
ccugprof_sub

,OPEID,CCUGPROF
1,100200,10.0
5,105200,9.0
6,2503400,5.0
11,105500,15.0
13,100500,10.0
...,...,...
19794,4238601,-2.0
19795,3889301,-2.0
19796,4162401,-2.0
19797,3915304,-2.0


In [18]:
# Checks for duplicates in CCUSIZSET
print("NUM OF DUPLICATES IDs: " + str(ccsizset_sub.duplicated(subset=['OPEID']).sum()))
print("NUM OF DUPLICATE ROWS: " + str(ccsizset_sub.duplicated().sum()))

NUM OF DUPLICATES IDs: 25
NUM OF DUPLICATE ROWS: 0


In [19]:
# Gets a temporary subset of all duplicates
ccsizset_dup = ccsizset_sub[ccsizset_sub.duplicated(subset=['OPEID'], keep=False)]
ccsizset_dup.sort_values(['OPEID'])

,OPEID,CCSIZSET
1937,145900,6.0
17072,145900,12.0
2044,146900,14.0
18091,146900,9.0
2013,147900,13.0
13872,147900,12.0
2075,153500,16.0
18732,153500,9.0
19790,180500,-2.0
3298,180500,12.0


In [20]:
# Mimic CCUGPROF because the duplicates are the same
ccsizset_sub.drop_duplicates(subset=['OPEID'], inplace=True)
ccsizset_sub

,OPEID,CCSIZSET
1,100200,14.0
5,105200,15.0
6,2503400,6.0
11,105500,12.0
13,100500,14.0
...,...,...
19794,4238601,-2.0
19795,3889301,-2.0
19796,4162401,-2.0
19797,3915304,-2.0


In [21]:
# All remaining duplicates can be dealt through averaging
adm_rate_sub = adm_rate_sub.groupby('OPEID').mean().reset_index()
display(adm_rate_sub)
sat_avg_sub = sat_avg_sub.groupby('OPEID').mean().reset_index()
display(sat_avg_sub)
tut_in_sub = tut_in_sub.groupby('OPEID').mean().reset_index()
display(tut_in_sub)
tut_out_sub = tut_out_sub.groupby('OPEID').mean().reset_index()
display(tut_out_sub)
comp_yr4_sub = comp_yr4_sub.groupby('OPEID').mean().reset_index()
display(comp_yr4_sub)
comp_lyr4_sub = comp_lyr4_sub.groupby('OPEID').mean().reset_index()
display(comp_lyr4_sub)

,OPEID,ADM_RATE
0,100200,0.891700
1,100300,0.495533
2,100400,0.594500
3,100500,0.805033
4,100900,0.799667
...,...,...
2125,4266000,0.740200
2126,4267800,0.809100
2127,4269300,0.585400
2128,4271200,0.805600


,OPEID,SAT_AVG
0,100200,911.666667
1,100300,1024.000000
2,100400,1127.333333
3,100500,910.666667
4,100900,1262.666667
...,...,...
1380,4244300,1248.000000
1381,4248500,956.666667
1382,4251400,1100.000000
1383,4254600,1048.000000


,OPEID,TUITIONFEE_IN
0,100200,9655.666667
1,100300,20930.000000
2,100400,12400.000000
3,100500,10452.000000
4,100700,4520.000000
...,...,...
4217,52098843,9900.000000
4218,52098845,9900.000000
4219,82098816,9749.666667
4220,82098821,9840.000000


,OPEID,TUITIONFEE_OUT
0,100200,18028.666667
1,100300,20930.000000
2,100400,25040.000000
3,100500,18316.000000
4,100700,8170.000000
...,...,...
3942,52098843,9900.000000
3943,52098845,9900.000000
3944,82098816,9749.666667
3945,82098821,9840.000000


,OPEID,C150_4
0,100200,0.261000
1,100300,0.298100
2,100400,0.497833
3,100500,0.264500
4,100900,0.765700
...,...,...
2515,72098885,0.000000
2516,82098816,0.168367
2517,82098821,0.142700
2518,82098842,0.180600


,OPEID,C150_L4
0,100700,0.258967
1,101300,0.217167
2,101500,0.177133
3,101700,0.191167
4,101800,0.230967
...,...,...
3870,4276000,0.684200
3871,4276500,0.904800
3872,4276700,0.882400
3873,4276800,0.800000


In [22]:
# Merge everything together!!!!
merged_sub_temp1 = pd.merge(name_sub, oper_sub, on='OPEID', how='outer')
merged_sub_temp2 = pd.merge(locale_sub, control_sub, on='OPEID', how='outer')
merged_sub_temp3 = pd.merge(ccugprof_sub, ccsizset_sub, on='OPEID', how='outer')
merged_sub_temp4 = pd.merge(adm_rate_sub, sat_avg_sub, on='OPEID', how='outer')
merged_sub_temp5 = pd.merge(tut_in_sub, tut_out_sub, on='OPEID', how='outer')
merged_sub_temp6 = pd.merge(comp_yr4_sub, comp_lyr4_sub, on='OPEID', how='outer')

# Second level of merge
merged_sub_temp1 = pd.merge(merged_sub_temp1, merged_sub_temp2, on='OPEID', how='outer')
merged_sub_temp2 = pd.merge(merged_sub_temp3, merged_sub_temp4, on='OPEID', how='outer')
merged_sub_temp3 = pd.merge(merged_sub_temp5, merged_sub_temp6, on='OPEID', how='outer')

# Third level of merge
merged_sub_temp1 = pd.merge(merged_sub_temp1, merged_sub_temp2, on='OPEID', how='outer')

# Last level of merge
merged_sub = pd.merge(merged_sub_temp1, merged_sub_temp3, on='OPEID', how='outer')

merged_sub

,OPEID,INSTNM,CURROPER,LOCALE,CONTROL,CCUGPROF,CCSIZSET,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,C150_4,C150_L4
0,100200,Alabama A & M University,1.0,12.0,1,10.0,14.0,0.891700,911.666667,9655.666667,18028.666667,0.261000,NaN
1,105200,University of Alabama at Birmingham,1.0,12.0,1,9.0,15.0,0.806867,1180.000000,8312.000000,19034.666667,0.547667,NaN
2,2503400,Amridge University,1.0,12.0,2,5.0,6.0,NaN,NaN,6993.333333,6993.333333,0.277267,NaN
3,105500,University of Alabama in Huntsville,1.0,12.0,1,15.0,12.0,0.794600,1297.666667,10278.666667,21484.666667,0.497200,NaN
4,100500,Alabama State University,1.0,12.0,1,10.0,14.0,0.805033,910.666667,10452.000000,18316.000000,0.264500,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7545,157115,Georgia Military College - Eastman,1.0,NaN,1,NaN,NaN,NaN,NaN,6120.000000,NaN,NaN,NaN
7546,4204801,American College of Barbering - Florence,1.0,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7547,4221501,BCI - Malden,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7548,4182601,HCI College - Fort Lauderdale Campus,1.0,NaN,3,NaN,NaN,NaN,NaN,21997.500000,NaN,NaN,NaN


In [23]:
# Prints number of duplicate OPEIDs
print("NUM OF DUPLICATES IDs: " + str(merged_sub.duplicated(subset=['OPEID']).sum()))

NUM OF DUPLICATES IDs: 0


Now that we have one merged subset, we can keep the operational colleges and assess the quality of the values after that!

In [24]:
# Returns the number of operational, non-operational, and unknown colleges
merged_sub['CURROPER'].value_counts(dropna=False)

1.0    6538
NaN     774
0.0     238
Name: CURROPER, dtype: int64

In [25]:
# Makes a subset with only operational colleges
data_sub = merged_sub.loc[merged_sub['CURROPER'] == 1.0]
data_sub

,OPEID,INSTNM,CURROPER,LOCALE,CONTROL,CCUGPROF,CCSIZSET,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,C150_4,C150_L4
0,100200,Alabama A & M University,1.0,12.0,1,10.0,14.0,0.891700,911.666667,9655.666667,18028.666667,0.261000,NaN
1,105200,University of Alabama at Birmingham,1.0,12.0,1,9.0,15.0,0.806867,1180.000000,8312.000000,19034.666667,0.547667,NaN
2,2503400,Amridge University,1.0,12.0,2,5.0,6.0,NaN,NaN,6993.333333,6993.333333,0.277267,NaN
3,105500,University of Alabama in Huntsville,1.0,12.0,1,15.0,12.0,0.794600,1297.666667,10278.666667,21484.666667,0.497200,NaN
4,100500,Alabama State University,1.0,12.0,1,10.0,14.0,0.805033,910.666667,10452.000000,18316.000000,0.264500,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7544,157114,Georgia Military College - Dublin,1.0,NaN,1,NaN,NaN,NaN,NaN,6120.000000,NaN,NaN,NaN
7545,157115,Georgia Military College - Eastman,1.0,NaN,1,NaN,NaN,NaN,NaN,6120.000000,NaN,NaN,NaN
7546,4204801,American College of Barbering - Florence,1.0,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7548,4182601,HCI College - Fort Lauderdale Campus,1.0,NaN,3,NaN,NaN,NaN,NaN,21997.500000,NaN,NaN,NaN


In [26]:
# Gets a list of nulls for quality assurance
data_sub.isnull().sum()

OPEID                0
INSTNM               0
CURROPER             0
LOCALE             432
CONTROL              0
CCUGPROF           432
CCSIZSET           432
ADM_RATE          4485
SAT_AVG           5177
TUITIONFEE_IN     2730
TUITIONFEE_OUT    2957
C150_4            4255
C150_L4           3252
dtype: int64

In [27]:
# Makes sure none of Rutgers Newark values are null
newark_data = data_sub.loc[data_sub['OPEID'] == 262902]
newark_data

,OPEID,INSTNM,CURROPER,LOCALE,CONTROL,CCUGPROF,CCSIZSET,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,C150_4,C150_L4
2185,262902,Rutgers University-Newark,1.0,11.0,1,13.0,16.0,0.640967,1065.333333,14107.666667,30074.333333,0.661133,NaN


## Quality Assessment ##

The first issue when conducting the null test is that a lot of the data is null. However, with the data that does exist, we can average them out and the only we can do this is because the data comes from reputable government sources like Federal Student Aid and the Integrated Postsecondary Education Data System. With all this being said, we will have to drop the C150_L4 column because for Rutgers - Newark, that value is null. We have no way to compare it with other colleges if that value is null. We must also note that the reason for C150_L4 being null is that Rutgers - Newark is not a school that does only assosiates degrees.

In [28]:
data_sub = data_sub.drop(columns=['C150_L4']).reset_index(drop=True)
data_sub

,OPEID,INSTNM,CURROPER,LOCALE,CONTROL,CCUGPROF,CCSIZSET,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,C150_4
0,100200,Alabama A & M University,1.0,12.0,1,10.0,14.0,0.891700,911.666667,9655.666667,18028.666667,0.261000
1,105200,University of Alabama at Birmingham,1.0,12.0,1,9.0,15.0,0.806867,1180.000000,8312.000000,19034.666667,0.547667
2,2503400,Amridge University,1.0,12.0,2,5.0,6.0,NaN,NaN,6993.333333,6993.333333,0.277267
3,105500,University of Alabama in Huntsville,1.0,12.0,1,15.0,12.0,0.794600,1297.666667,10278.666667,21484.666667,0.497200
4,100500,Alabama State University,1.0,12.0,1,10.0,14.0,0.805033,910.666667,10452.000000,18316.000000,0.264500
...,...,...,...,...,...,...,...,...,...,...,...,...
6533,157114,Georgia Military College - Dublin,1.0,NaN,1,NaN,NaN,NaN,NaN,6120.000000,NaN,NaN
6534,157115,Georgia Military College - Eastman,1.0,NaN,1,NaN,NaN,NaN,NaN,6120.000000,NaN,NaN
6535,4204801,American College of Barbering - Florence,1.0,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6536,4182601,HCI College - Fort Lauderdale Campus,1.0,NaN,3,NaN,NaN,NaN,NaN,21997.500000,NaN,NaN


In [29]:
# Gets data types
data_sub.dtypes

OPEID               int64
INSTNM             object
CURROPER          float64
LOCALE            float64
CONTROL             int64
CCUGPROF          float64
CCSIZSET          float64
ADM_RATE          float64
SAT_AVG           float64
TUITIONFEE_IN     float64
TUITIONFEE_OUT    float64
C150_4            float64
dtype: object

In [30]:
data_sub.describe()

,OPEID,CURROPER,LOCALE,CONTROL,CCUGPROF,CCSIZSET,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,C150_4
count,6.538000e+03,6538.0,6106.000000,6538.000000,6106.000000,6106.000000,2053.000000,1361.000000,3808.000000,3581.000000,2283.000000
mean,1.758864e+06,1.0,20.002948,2.067452,3.772683,4.623321,0.676874,1109.260593,15047.359900,18438.196128,0.500976
std,2.661812e+06,0.0,9.668798,0.835869,5.840053,6.445686,0.198959,131.444833,12979.157022,12228.650801,0.209726
min,1.002000e+05,1.0,-3.000000,1.000000,-2.000000,-2.000000,0.000000,735.000000,480.000000,480.000000,0.000000
25%,3.072250e+05,1.0,12.000000,1.000000,-2.000000,-2.000000,0.560967,1023.666667,5056.000000,8678.666667,0.345317
50%,1.005550e+06,1.0,21.000000,2.000000,2.000000,3.000000,0.701000,1090.000000,10746.000000,15135.000000,0.500000
75%,3.027275e+06,1.0,22.000000,3.000000,10.000000,10.000000,0.824600,1170.666667,20372.000000,25416.666667,0.649783
max,8.209888e+07,1.0,43.000000,3.000000,15.000000,18.000000,1.000000,1559.666667,73636.666667,73636.666667,1.000000


In [31]:
# Locale Mode
data_sub['LOCALE'].value_counts()

 21.0    1540
 11.0    1393
 13.0     789
 12.0     691
 32.0     416
 41.0     341
 33.0     317
 22.0     183
 23.0     127
 31.0     124
 42.0     114
 43.0      61
-3.0       10
Name: LOCALE, dtype: int64

In [32]:
# Control Mode
data_sub['CONTROL'].value_counts()

3    2519
1    2078
2    1941
Name: CONTROL, dtype: int64

In [33]:
# CCUGPROF Mode
data_sub['CCUGPROF'].value_counts()

-2.0     2151
 1.0      542
 11.0     422
 5.0      359
 13.0     331
 2.0      327
 0.0      294
 14.0     293
 4.0      276
 7.0      239
 10.0     214
 12.0     209
 3.0      144
 9.0      136
 15.0     133
 6.0       32
 8.0        4
Name: CCUGPROF, dtype: int64

In [34]:
# CCSIZSET Mode
data_sub['CCSIZSET'].value_counts()

-2.0     2151
 6.0      489
 2.0      423
 11.0     376
 1.0      360
 3.0      302
 18.0     271
 8.0      268
 12.0     191
 9.0      175
 13.0     174
 14.0     166
 10.0     159
 4.0      144
 15.0     132
 16.0     128
 7.0       90
 5.0       64
 17.0      43
Name: CCSIZSET, dtype: int64

## Data For Comparison ##

Here is the averages or modes for each column
    1. Locale
        a. Top 3 include -> 21, 11, 13
    2. Control
        a. 3 is most common
    3. CCUGPROF
        a. Top 3 include (ignoring unknowns) -> 1.0, 11.0, 5.0
    4. CCSIZSET
        a. Top 3 include (ignoring unknowns) -> 6.0, 2.0, 11.0
    5. ADM_RATE
        a. Mean -> 0.68
        b. 25% -> 0.56
        c. 50% -> 0.70
        d. 75% -> 0.82
    6. SAT_AVG
        a. Mean -> 1109.26
        b. 25% -> 1023.67
        c. 50% -> 1090.00
        d. 75% -> 1170.67
    7. TUITIONFEE_IN
        a. Mean -> 15047.36
        b. 25% -> 5056.00
        c. 50% -> 10746.00
        d. 75% -> 20372.00
    8. TUITIONFEE_OUT
        a. Mean -> 18438.20
        b. 25% -> 8678.67
        c. 50% -> 15135.00
        d. 75% -> 25416.67
    9. C150_4
        a. Mean -> 0.50
        b. 25% -> 0.35
        c. 50% -> 0.50
        d. 75% -> 0.65

In [35]:
# Print the data out for Rutgers - Newark
newark_data

,OPEID,INSTNM,CURROPER,LOCALE,CONTROL,CCUGPROF,CCSIZSET,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,C150_4,C150_L4
2185,262902,Rutgers University-Newark,1.0,11.0,1,13.0,16.0,0.640967,1065.333333,14107.666667,30074.333333,0.661133,NaN


## Final Analysis ## 
After looking at data, we notice that there are a lot of points like CONTROL, CCUGPROF, CCSIZSET, TUITIONFEE_OUT, and C150_4 where Rutgers University - Newark is far from ordinary. Out of the 9 different testable columns, 5 of them are not average for Rutgers - Newark. That is more than half. The null hypothesis for this analysis turned out to be false. With further time and analysis we can better assess if Rutgers - Newark is average with factors like demographics and family income put into consideration. A further step could also include assessing if Rutgers Newark is a better investment than other colleges as the use case is for applying to colleges. No one wants to do average investments, they want the best investment.